# RSS: Expansión

## 1. Import Libraries

In [1]:
import feedparser
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import googletrans
import datetime
from termcolor import colored
from IPython.display import display, Markdown, Latex, HTML
import re
from IPython.display import Image

In [2]:
def get_only_text(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    # Eliminar Última Hora
    rest = soup.find_all('p', attrs={'class' : "line-clamp_x2 ue-c-widget__article-headline"})[0].text
    index = text.find(rest)
    final_text = text[:index]
    return [final_text,soup.title.text]

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [3]:
def expansion_news(expansion_entries,word_count=100):
    expansion = {}
    for i in range(len(expansion_entries)):
        noticia = {'newspaper':None,'title':None,'headline':None,'summarise':None,'date':None,'link':None,'text':None,'current_date':None,'img':None,'premium':None}
        # Periodico
        noticia['newspaper'] = find_between(s=expansion_entries[i]['id'], first='www.', last='.' )
        # Title
        noticia['title'] = expansion_entries[i]['title']
        # Headline
        index = expansion_entries[i]['summary_detail']['value'].find('&nbsp')
        if index != -1:
            noticia['headline'] = expansion_entries[i]['summary_detail']['value'][:index]
        else:
            noticia['headline'] = re.sub(pattern='&quot;',repl='"',string=expansion_entries[i]['summary_detail']['value'])
        # Date
        noticia['date'] = {'date1':expansion_entries[i]['published'],'date2':expansion_entries[i]['published_parsed']}
        # Link
        noticia['link'] = expansion_entries[i]['id']
        # Text
        text = get_only_text(expansion_entries[i]['id'])[0]
        noticia['text'] = text
        # Summarise
        if text.find('Para seguir leyendo') == -1:
            noticia['summarise'] = summarize(text, word_count=word_count)
            noticia['premium'] = False
        else:
            noticia['summarise'] = 'Premium Content'
            noticia['premium'] = True
        # Current Date
        noticia['current_date'] = datetime.datetime.now().timetuple()
        # Image
        noticia['img'] = expansion_entries[i]['media_content'][0]['url']
        expansion.update({'Noticia'+'_'+str("%02d")%(i+1):noticia})
    return expansion

## 2. RSS Expansión

In [4]:
url_expansion = 'https://e00-expansion.uecdn.es/rss/empresasbanca.xml'
expansion = feedparser.parse(url_expansion)
expansion_entries = expansion['entries']

print('Total Entradas Expansión:',len(expansion_entries))

Total Entradas Expansión: 35


In [5]:
expansion = expansion_news(expansion_entries,word_count=100)

In [6]:
yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
before_yes = datetime.datetime.today() - datetime.timedelta(days=2)

for i in list(expansion.keys()):
    if expansion[i]['date']['date2'][2] in [datetime.datetime.today().day,yesterday.day,before_yes.day]:
        print(colored((expansion[i]['newspaper']).upper(),'green',attrs=['bold']))
        print(colored(expansion[i]['date']['date1'],'grey',attrs=['bold']))
        print(colored(expansion[i]['title'],'magenta',attrs=['bold']))
        print(colored('Headline','blue'))
        print(expansion[i]['headline'])
        print(colored('Summarise','blue'))
        
        if expansion[i]['summarise'] != 'Premium Content':
            print(expansion[i]['summarise'])
        else:
            print(colored(expansion[i]['summarise'],'red'))
        print(colored('Text','blue'))
        print(expansion[i]['text'])
        url = expansion[i]['img']
        display(Image(url=url,width=500))
        display(Markdown('<a href="'+expansion[i]['link']+'">Link</a>'))
        print('\n')

EXPANSION
Wed, 08 Jul 2020 18:17:32 +0200
Bankinter coloca 350 millones en bonos convertibles con un cupón del 6,25%
Headline
Bankinter ha colocado una oferta de valores perpetuos eventualmente convertibles en acciones ordinarias de la entidad por un importe máximo de 350 millones de euros, que devengarán un cupón trimestral del 6,25%.
Summarise
Bankinter ha colocado una oferta de valores perpetuos eventualmente convertibles en acciones ordinarias de la entidad por un importe máximo de 350 millones de euros, que devengarán un cupón trimestral del 6,25%.
La alta demanda de esta emisión de deuda AT1, conocida como 'cocos' en la jerga financiera, que ha superado en 2,2 veces la oferta, con 800 millones de euros, ha permitido abaratar el precio en 25 puntos básicos del 6,5% al 6,25%.
Bankinter, Barclays, BBVA, Credit Agricole CIB y Natixis actúan como entidades colocadoras para esta emisión.
Text
Bankinter ha colocado una oferta de valores perpetuos eventualmente convertibles en acciones o

<a href="https://www.expansion.com/empresas/banca/2020/07/08/5f05903c468aeb65678b457e.html">Link</a>



EXPANSION
Wed, 08 Jul 2020 13:34:59 +0200
Nickel, el banco digital de BNP Paribas, aspira a alcanzar 700.000 cuentas en España en 2024
Headline
BNP Paribas ha presentado este miércoles Nickel España, el banco digital que lanzará "a finales de verano" siguiendo el modelo implantado en Francia, basado en la colaboración con loteros, y que aspira a alcanzar 700.000 cuentas en los próximos cuatro años.
Summarise
Nickel permitirá al cliente abrir en 5 minutos una cuenta corriente en cualquier establecimiento de loterías y estancos asociados sin ninguna condición de ingresos, y ofrecerá servicios básicos como una tarjeta vinculada para pagar en todo el mundo, realizar compras "online" y retirar dinero en efectivo.
Si bien dispondrá de un servicio telefónico de atención asentado en España, el cliente realizará sus operaciones físicas en los establecimientos adheridos, los cuales "dan la seguridad de que se cumplirán con los requisitos" al ser puntos de venta que, por su naturaleza, "cuentan

<a href="https://www.expansion.com/empresas/banca/2020/07/08/5f05af58468aeb7a128b45d3.html">Link</a>



EXPANSION
Wed, 08 Jul 2020 13:26:50 +0200
Las reclamaciones a la banca caen un 26% al haber menos quejas hipotecarias
Headline
El Banco de España recibió 14.638 reclamaciones en 2019, un 25,7% menos que un año antes, principalmente por la caída de las quejas relacionadas con hipotecas, sin embargo, registró "un aumento notable" de las disputas de clientes con sus entidades por tarjetas.
Summarise
El Banco de España recibió 14.638 reclamaciones en 2019, un 25,7% menos que un año antes, principalmente por la caída de las quejas relacionadas con hipotecas, sin embargo, registró "un aumento notable" de las disputas de clientes con sus entidades por tarjetas.
En el caso de las tarjetas, algunas reclamaciones fueron por las condiciones de comercialización de las denominadas "revolving", que permiten aplazar el pago de las compras, pero también hubo quejas por fraudes informáticos ante el creciente uso del pago con tarjetas en comercios electrónicos, explican fuentes desde el Banco de Españ

...

<a href="https://www.expansion.com/empresas/banca/2020/07/06/5f02ba1fe5fdeab5138b45c8.html">Link</a>



EXPANSION
Mon, 06 Jul 2020 08:42:51 +0200
Horta-Osório dejará su cargo como consejero delegado de Lloyds el próximo año
Headline
El banquero portugués António Horta-Osório ha comunicado al consejo de administración de LLoyds Banking Group su intención de abandonar en 2021 el puesto de consejero delegado de la entidad, que ocupaba desde marzo de 2011, según ha confirmado el banco británico.
Summarise
El banquero portugués António Horta-Osório ha comunicado al consejo de administración de LLoyds Banking Group su intención de abandonar en 2021 el puesto de consejero delegado de la entidad, que ocupaba desde marzo de 2011, según ha confirmado el banco británico.
"Por supuesto, con emociones encontradas, anuncio mi intención de renunciar como consejero delegado de Lloyds Banking Group para junio del próximo año (...) Me ha honrado desempeñar mi parte en la transformación de gran parte de nuestro negocio.
Por otro lado, al margen de la salida de Horta-Osório, el consejo de administración d

<a href="https://www.expansion.com/empresas/banca/2020/07/06/5f02c7b5e5fdea791f8b45b2.html">Link</a>



EXPANSION
Mon, 06 Jul 2020 07:52:38 +0200
CaixaBank estrena plataforma para atraer clientes extranjeros
Headline
CaixaBank ha puesto en marcha una plataforma digital para facilitar la captación de clientes extranjeros.
Summarise
Premium Content
Text
CaixaBank ha puesto en marcha una plataforma digital para facilitar la captación de clientes extranjeros. Se trata de una aplicación que permite a los no residentes abrir rápidamente una cuenta en HolaBank -el programa dirigido a sus clientes internacionales en España- y también solicitar una hipoteca desde sus países de origen e íntegramente de forma di Para seguir leyendo, hágase Premium ¿Ya es Premium? Inicie sesión Mensual primer mes después 14,99€   Anual primer mes después 7,50€/mes (Pago único de 90€)
 4 meses gratis  Descubra qué es Premium   Cancele cuando quiera    Consulte los términos y condiciones del servicio  


<a href="https://www.expansion.com/empresas/banca/2020/07/06/5f02bbf6e5fdeac4678b45dc.html">Link</a>